<a href="https://colab.research.google.com/github/tonyscan6003/Exercisescripts/blob/master/RE4017_lab2_2_keypoint_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Feature Matching with ORB

In this lab we will test out feature matching with the ORB Feature Detector & Descriptor. ORB is comprised of the FAST corner detector and the BRIEF binary descriptor. 

![Feature Matching with ORB](https://github.com/tonyscan6003/RE4017/blob/master/images/orb_match.JPG?raw=true)

Read More about ORB at:
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_orb/py_orb.html
https://medium.com/analytics-vidhya/introduction-to-orb-oriented-fast-and-rotated-brief-4220e8ec40cf

In [0]:
import numpy as np
import urllib
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
from mpl_toolkits import mplot3d
from scipy import signal

# Read URL Links.
def url_to_image(url):
	resp = urllib.request.urlopen(url)
	temp_image = np.asarray(bytearray(resp.read()), dtype="uint8")
	temp_image = cv2.imdecode(temp_image, cv2.IMREAD_COLOR)
	temp_image = cv2.cvtColor(temp_image, cv2.COLOR_BGR2RGB) # OpenCV defaults to BGR, but we need RGB here..
	return temp_image

# List of item urls:
# Object
book_url = "https://github.com/tonyscan6003/RE4017/blob/master/images/IMG_20200228_155300.jpg?raw=true"
# Book test
book_test_url1 ="https://github.com/tonyscan6003/RE4017/blob/master/images/IMG_20200228_155346.jpg?raw=true"
book_test_url2 = "https://github.com/tonyscan6003/RE4017/blob/master/images/IMG_20200228_155432.jpg?raw=true"

# Object
house_url = "https://github.com/tonyscan6003/RE4017/blob/master/images/IMG_20200305_144233.jpg?raw=true"
# House Test Images
house_test_url2 = "https://github.com/tonyscan6003/RE4017/blob/master/images/IMG_20200305_144436.jpg?raw=true"
house_test_url1 = "https://github.com/tonyscan6003/RE4017/blob/master/images/IMG_20200305_144324.jpg?raw=true"
house_test_url3 = "https://github.com/tonyscan6003/RE4017/blob/master/images/IMG_20200305_144341.jpg?raw=true"

In the cells below we will try identifying various object from a cluttered scene using feature matching. 

The reference image (i.e object to be found) can be changed to either of the listed items above. 

There is also a choice of test images to try, with the objects rearranged.


In [0]:
ref_image_url = book_url
test_image_url = book_test_url1
# adjust scale factor
scale_factor = 4

In [0]:
#Read in images
ref_image = url_to_image(ref_image_url)
test_image = url_to_image(test_image_url)

# find size of images
ref_size = np.shape(ref_image)
test_size = np.shape(test_image) 

# apply scaling
ref_image = cv2.resize(ref_image, dsize=(int(ref_size[1]/scale_factor),int(ref_size[0]/scale_factor)), interpolation=cv2.INTER_CUBIC)
test_image = cv2.resize(test_image, dsize=(int(test_size[1]/scale_factor),int(test_size[0]/scale_factor)), interpolation=cv2.INTER_CUBIC)

# Plot reference and test images
f, axarr = plt.subplots(1,2,figsize=(10,20))
axarr[0].imshow(ref_image)
axarr[0].axis('off') 
axarr[0].title.set_text('Reference Image')
axarr[1].imshow(test_image)
axarr[1].axis('off') 
axarr[1].title.set_text('Test Image')


#Feature Detection & Descriptor
The ORB Descriptor includes the FAST Corner detector as its feature detector. We will display the found keypoints in the reference image, showing the keypoints with scale information and also just the keypoint locations.

Given the feature detector is Corner detector, are the keypoints being found where you expect?

The ORB Descriptor has been developed for use within the OpenCV python package. As can be seen in the code below it is very simple to initiate (use: ORB_create) the ORB descriptor and find the keypoints and their associated descriptors (use: detectANDCompute)

In [0]:
# Convert images to grey scale
img1 = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

# Initiate ORB detector
orb = cv2.ORB_create(nfeatures=1000, scoreType=cv2.ORB_FAST_SCORE)
# find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)



Cell Below plots found keypoints in reference image.

In [0]:

keypoints_without_size = np.copy(ref_image)
testkp_without_size = np.copy(test_image)
keypoints_with_size = np.copy(ref_image)

cv2.drawKeypoints(ref_image, kp1, keypoints_without_size, color = (0, 255, 0))
cv2.drawKeypoints(test_image, kp2, testkp_without_size, color = (0, 255, 0))
cv2.drawKeypoints(ref_image, kp1, keypoints_with_size, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


# Display image with and without keypoints size
fx, plots = plt.subplots(1, 3, figsize=(20,10))

plots[0].set_title("Reference Image keypoints With Scale")
plots[0].imshow(keypoints_with_size, cmap='gray')

plots[1].set_title("Reference Image keypoints Location Only")
plots[1].imshow(keypoints_without_size, cmap='gray')

plots[2].set_title("Test Image keypoints Location Only")
plots[2].imshow(testkp_without_size, cmap='gray')

# Print the number of keypoints detected in the training image
print("Number of Keypoints Detected In The Training Image: ", len(kp1))

# Print the number of keypoints detected in the query image
print("Number of Keypoints Detected In The Query Image: ", len(kp2))

#Perform Feature Matching
Now that we have demonstrated finding the keypoints, we will perform matching between the keypoints of the reference and test images. The matching is performed by brute force. We will initiate the OpenCV BFMatcher and use the match command to perform matching.  Each reference image keypoint is compared against all the test image keypoints using a distance metric. The minimum distance corresponds to a match. We will dislay the best n (usually 10-20) keypoints. 

In [0]:
# Set the number of "Best Matches to Display"
n = 30

In [0]:

#feature matching
# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# Match descriptors.
matches = bf.match(des1,des2)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw first n matches.
img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches[:n],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(20,20))
plt.title('Best Matching Points')
plt.imshow(img3),plt.show()


print("\nNumber of Matching Keypoints Between The Training and Query Images: ", len(matches))